# Asset Return Prediction with Facebook Prophet Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly
from datetime import timedelta

c:\Users\matheuscn.ELE\.conda\envs\st\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
returns = pd.read_csv("../data/weekly_returns.csv")

returns.rename(columns={"Date":"ds"},inplace=True)
returns.ds = pd.DatetimeIndex(returns.ds)

returns.head()

,ds,ABEV3.SA,BBAS3.SA,BBDC4.SA,CPLE6.SA,CSAN3.SA,CSNA3.SA,ELET3.SA,ENBR3.SA,GGBR4.SA,...,JBSS3.SA,PETR4.SA,SANB11.SA,SULA11.SA,TIMS3.SA,USIM5.SA,VALE3.SA,VIVT3.SA,WEGE3.SA,Selic
0,2010-01-04,3.857942,2.960869,1.851944,3.214508,-5.466719,3.975702,5.416819,5.699117,1.978909,...,4.833852,0.706120,-1.900820,-1.004999,2.758804,4.236805,8.025924,-2.721604,5.461178,0.001647
1,2010-01-11,1.956319,-1.929901,-2.641365,-1.448335,-1.019157,0.503320,-6.741704,4.716572,-2.363506,...,1.692414,-3.301529,-2.373370,-3.036314,-1.369902,-1.087498,-0.671246,-1.251025,-2.985252,0.001647
2,2010-01-18,0.012591,-2.457353,-12.690203,-1.066689,-2.288032,-7.474728,15.442856,-2.330498,-10.209203,...,-5.582358,-2.837063,-5.382457,3.923616,-0.692022,-7.855276,-5.657556,-0.665164,-4.401735,0.001647
3,2010-01-25,-5.826853,-1.553700,-1.426054,3.270325,-7.242973,2.578421,-4.779064,-1.960836,-2.763375,...,-2.537157,-1.683138,3.709170,-3.882837,-2.817087,6.453844,-3.218905,2.580398,0.277436,0.001647
4,2010-02-01,-3.045943,-0.714315,-4.171289,-3.431324,9.402891,-1.280900,-50.766704,-4.483725,-2.679601,...,-3.597110,-8.072616,-10.624915,-4.401520,-4.979089,-3.905943,-3.643590,-3.054321,-4.823606,0.001647


In [3]:
df_predictions = pd.DataFrame()
window = 12
for asset_name in returns.columns[1:]:
    asset = returns[["ds",asset_name]]
    asset.rename(columns={asset_name:"y"},inplace=True)

    last_train_year = 2020 
    test_year = 2021

    train = asset[asset.ds.dt.year <= 2019]
    test = asset[asset.ds.dt.year >= 2020]

    
    #Trainnig the model
    model = Prophet(growth="flat",
                n_changepoints = 5,
                weekly_seasonality=True,
                daily_seasonality=True,
                yearly_seasonality=True)
    model.add_country_holidays(country_name="Brazil")
    model.fit(train)

    #Creating in sample predictions dataframe
    future = model.make_future_dataframe(periods=1,freq='w')
    forecast = model.predict(future)
    df_prediction_train = pd.DataFrame(forecast[["ds","yhat"]].iloc[0:-1,:])
    df_prediction_train.set_index("ds",inplace=True)

    #Test Forecasting as Rolling Window
    days = test.ds
    dict_prediction_test = {}

    for i in range(window,len(days)+window):
    
        window_returns = asset[i-window:i]
        
        future = model.make_future_dataframe(periods=i+1-window,freq="w")
        prediction = model.predict(future)
        d = pd.to_datetime(future.iloc[-1].values[0])+timedelta(days=1)
        dict_prediction_test[str(d)] = prediction.yhat.iloc[-1]


    df_prediction_test = pd.DataFrame(index=dict_prediction_test.keys(),data=dict_prediction_test.values())                            
    df_prediction_test.index = pd.to_datetime(df_prediction_test.index)
    df_prediction_test.rename(columns={0:"yhat"},inplace=True) 
    
    df_prediction_asset = pd.concat([df_prediction_train,df_prediction_test],axis=0)
    df_prediction_asset.rename(columns={"yhat":asset_name},inplace=True)

    df_predictions[asset_name] = df_prediction_asset
    print("%s Finished"%asset_name)
    


df_predictions.to_csv("../data/predicted_returns_prophet.csv")
df_predictions

C:\Users\matheuscn.ELE\AppData\Local\Temp\ipykernel_14976\3539546350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset.rename(columns={asset_name:"y"},inplace=True)
11:30:12 - cmdstanpy - INFO - Chain [1] start processing
11:30:13 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\matheuscn.ELE\AppData\Local\Temp\ipykernel_14976\3539546350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset.rename(columns={asset_name:"y"},inplace=True)
11:30:35 - cmdstanpy - INFO - Chain [1] start processing
11:30:35 - cmdstanpy - INFO - Chain [1] done processing


ABEV3.SA Finished


C:\Users\matheuscn.ELE\AppData\Local\Temp\ipykernel_14976\3539546350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset.rename(columns={asset_name:"y"},inplace=True)
11:31:18 - cmdstanpy - INFO - Chain [1] start processing
11:31:18 - cmdstanpy - INFO - Chain [1] done processing


BBAS3.SA Finished


C:\Users\matheuscn.ELE\AppData\Local\Temp\ipykernel_14976\3539546350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset.rename(columns={asset_name:"y"},inplace=True)
11:31:57 - cmdstanpy - INFO - Chain [1] start processing
11:31:57 - cmdstanpy - INFO - Chain [1] done processing


...

C:\Users\matheuscn.ELE\AppData\Local\Temp\ipykernel_14976\3539546350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset.rename(columns={asset_name:"y"},inplace=True)
11:40:35 - cmdstanpy - INFO - Chain [1] start processing
11:40:35 - cmdstanpy - INFO - Chain [1] done processing


VIVT3.SA Finished


C:\Users\matheuscn.ELE\AppData\Local\Temp\ipykernel_14976\3539546350.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset.rename(columns={asset_name:"y"},inplace=True)
11:41:21 - cmdstanpy - INFO - Chain [1] start processing
11:41:21 - cmdstanpy - INFO - Chain [1] done processing


WEGE3.SA Finished
Selic Finished


,ABEV3.SA,BBAS3.SA,BBDC4.SA,CPLE6.SA,CSAN3.SA,CSNA3.SA,ELET3.SA,ENBR3.SA,GGBR4.SA,ITUB4.SA,JBSS3.SA,PETR4.SA,SANB11.SA,SULA11.SA,TIMS3.SA,USIM5.SA,VALE3.SA,VIVT3.SA,WEGE3.SA,Selic
2010-01-04,0.849030,0.745534,1.601662,0.272795,0.504013,1.597308,2.242127,0.688020,0.285143,0.856814,-0.282312,0.709966,1.017560,0.495779,0.982110,-0.571137,0.510134,0.343117,0.328109,0.001715
2010-01-11,0.371238,-0.232434,0.317320,0.907605,0.069675,-0.639198,1.095701,0.319027,-1.213191,0.168773,0.551684,-0.090821,0.174789,0.104377,0.676421,-2.665148,-0.782951,0.165940,-0.160874,0.001800
2010-01-18,0.239241,-0.551994,-0.764049,0.974258,0.187437,-1.497824,-0.141232,0.034223,-1.256302,-0.257630,1.177008,-1.353265,-0.339051,0.315593,0.498693,-2.615929,-1.549119,0.499738,-0.719162,0.001892
2010-01-25,0.501072,0.041014,-0.799550,0.483918,0.731576,-0.631776,-0.538677,0.050769,0.120770,-0.063818,0.797390,-1.953317,-0.004126,0.799862,0.437932,-1.122961,-0.913239,0.820136,-0.987414,0.001923
2010-02-01,0.757876,0.931694,0.044012,-0.068572,1.105701,0.616494,-0.246891,0.189546,1.308332,0.551386,-0.157726,-1.227949,0.855558,0.909787,0.388900,-0.218899,0.619355,0.654698,-0.719282,0.001895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-29,0.286971,-0.146714,-0.436234,0.969352,-0.230074,-1.056597,-0.982852,0.201230,-0.528767,-0.285041,0.852950,-1.299478,0.294491,0.306341,-0.233094,-2.163094,0.080186,-0.825260,0.730320,0.001745
2021-12-06,-0.096370,-1.092245,-1.043864,0.687907,-0.388384,-1.982816,-1.427461,-0.046397,-1.523311,-0.704914,1.579369,-1.790425,-0.102679,-0.105927,0.208705,-2.840921,-0.133282,-0.474418,0.561964,0.001831
2021-12-13,0.122862,-0.872809,-0.704275,0.000884,0.024365,-0.886699,-1.220953,-0.059716,-0.884764,-0.434169,1.794018,-1.789037,-0.048421,0.110960,0.717077,-1.149993,-0.161985,0.414512,0.492038,0.001859
2021-12-20,0.750302,0.337425,0.583731,-0.569152,0.710087,1.621198,0.162123,0.291374,0.784270,0.396723,1.050111,-0.990289,0.617820,0.757767,1.125960,1.396821,0.406986,1.093972,0.532350,0.001803
